In [132]:
import pandas as pd
import numpy as np

In [133]:
trainingDataPpath = 'train.csv'
df = pd.read_csv(trainingDataPpath)

In [134]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [135]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [136]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [137]:
from sklearn.model_selection import train_test_split

In [138]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [139]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [140]:
type(y_test)

pandas.core.series.Series

In [141]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [142]:
X_test.isnull().sum()

Pclass       0
Sex          0
Age         37
SibSp        0
Parch        0
Fare         0
Embarked     0
dtype: int64

In [143]:
from sklearn.impute import SimpleImputer

In [144]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.fit_transform(X_test[['Age']])
X_test_embarked = si_embarked.fit_transform(X_test[['Embarked']])

In [145]:
from sklearn.preprocessing import OneHotEncoder

In [146]:
np.unique(X_train_embarked, return_counts=True)

(array(['C', 'Q', 'S'], dtype=object), array([125,  60, 527], dtype=int64))

In [147]:
X_train['Embarked'].value_counts()

Embarked
S    525
C    125
Q     60
Name: count, dtype: int64

In [148]:
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_ohe_sex = ohe_sex.fit_transform(X_train[['Sex']])
# X_train_ohe_embarked = ohe_embarked.fit_transform(X_train[['Embarked']])
X_train_ohe_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_ohe_sex = ohe_sex.fit_transform(X_test[['Sex']])
X_test_ohe_embarked = ohe_embarked.fit_transform(X_test[['Embarked']])

In [149]:
X_train_ohe_sex[:2]

array([[0., 1.],
       [0., 1.]])

In [150]:
X_train_ohe_embarked[:2]

array([[0., 0., 1.],
       [0., 0., 1.]])

In [151]:
X_train_rem = X_train.drop(columns=['Sex', 'Age', 'Embarked'])
X_test_rem = X_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [152]:
print(X_train_rem.shape)
print(X_train_age.shape)
print(X_train_ohe_sex.shape)
print(X_train_ohe_embarked.shape)

(712, 4)
(712, 1)
(712, 2)
(712, 3)


In [153]:
print(X_test_rem.shape)
print(X_test_age.shape)
print(X_test_ohe_sex.shape)
print(X_test_ohe_embarked.shape)

(179, 4)
(179, 1)
(179, 2)
(179, 3)


In [154]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_ohe_sex, X_train_ohe_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_ohe_sex, X_test_ohe_embarked), axis=1)

In [155]:
X_train_transformed.shape

(712, 10)

In [156]:
X_test_transformed.shape

(179, 10)

In [157]:
from sklearn.svm import SVC

In [158]:
clf = SVC(kernel='linear', C=1.0, random_state=42)
clf.fit(X_train_transformed, y_train)

SVC(kernel='linear', random_state=42)

In [159]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1], dtype=int64)

In [179]:
clf.score(X_test_transformed, y_test)

0.7821229050279329

In [162]:
from sklearn.metrics import accuracy_score

In [164]:
accuracy_score(y_test, y_pred)

0.7821229050279329

In [180]:
import pickle

In [182]:
pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('models/clf.pkl', 'wb'))